# Tictanic 분류 모델 구축
### 진행 단계:
1. 데이터 로딩
2. 결측치 확인
3. 결측치 처리 (Age 컬럼은 평균값으로 채우고, 나머지 결측치는 제거)
4. 레이블(생존 여부) 분포 확인 (불균형 데이터 확인)
5. 불필요한 컬럼 제거 (PassengerId, Name, Ticket, Cabin)
6. 범주형 변수 숫자 인코딩 (인코딩 전/후 레이블 분포 확인)
7. 5가지 분류 모델 (Random Forest, Decision Tree, Logistic Regression, KNN, SVM) 학습 및 평가

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [99]:
# 1. 데이터 로딩
url = "https://raw.githubusercontent.com/MyungKyuYi/AI-class/refs/heads/main/titanic.csv"
df = pd.read_csv(url)
print("=== 데이터 로딩 완료 ===")
print(df.head())

=== 데이터 로딩 완료 ===
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500  

In [101]:
# 2. 결측치 확인
print("=== 결측치 확인 ===")
print(df.isnull().sum())

=== 결측치 확인 ===
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [103]:
# 3. 결측치 처리
# Age 컬럼의 결측치를 평균값으로 채우기
mean_age = df['age'].mean()
df['Age'] = df['Age'].fillna(mean_age)
df = df.dropna()  # Age 외의 다른 결측치가 있는 행 제거
print("=== 결측치 처리 후 ===")
print(df.isnull().sum())

=== 결측치 처리 후 ===
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [109]:
# 4. 레이블 확인 (불균형 데이터 확인)
# Titanic 데이터셋에서는 'Survived'컬럼이 레이블로 사용됨
print("=== 레이블 분포 (변환 전) ===")
print(df['Survived'].value_counts())
# 레이블 컬럼명을 'label'로 변경
df.rename(columns={"Survived": "label"}, inplace=True)


=== 레이블 분포 (변환 전) ===
Survived
1    134
0     68
Name: count, dtype: int64


In [121]:
# 5. 불필요한 컬럼 제거
cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df = df.drop(columns=cols_to_drop, errors='ignore')
print("=== 불필요한 컬럼 제거 후 ===")
print(df.columns)

=== 불필요한 컬럼 제거 후 ===
Index(['label', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [113]:
# 6. 엔코딩 (숫자 변환)
print("=== 레이블 분포 (인코딩 전) ===")
print(df['label'].value_counts())

# 범주형 변수 (예: Sex, Embarked 등)를 숫자로 변환하기 위해 one-hot 인코딩 사용
df_encoded = pd.get_dummies(df, drop_first=True)

# 혹시 남아 있을 수 있는 결측치는 채워줍니다.
df_encoded = df_encoded.fillna(0)

print("=== 레이블 분포 (인코딩 후) ===")
print(df_encoded['label'].value_counts())


=== 레이블 분포 (인코딩 전) ===
label
1    134
0     68
Name: count, dtype: int64
=== 레이블 분포 (인코딩 후) ===
label
1    134
0     68
Name: count, dtype: int64


In [115]:
# 7. 분류 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# features와 target 분리
X = df_encoded.drop('label', axis=1)
y = df_encoded['label']

# 학습/테스트 세트 분할 (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 분류 모델 정의
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(random_state=42)
}

print("=== 분류 모델 평가 결과 ===")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"{name}: Accuracy = {acc:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("\n")

=== 분류 모델 평가 결과 ===
Random Forest: Accuracy = 0.7805
Confusion Matrix:
[[ 6  2]
 [ 7 26]]


Decision Tree: Accuracy = 0.8780
Confusion Matrix:
[[ 5  3]
 [ 2 31]]


Logistic Regression: Accuracy = 0.8293
Confusion Matrix:
[[ 5  3]
 [ 4 29]]


KNN: Accuracy = 0.7561
Confusion Matrix:
[[ 1  7]
 [ 3 30]]


SVM: Accuracy = 0.8049
Confusion Matrix:
[[ 0  8]
 [ 0 33]]


